In [1]:
import numpy as np
from keras.models import Sequential, load_model, Model
from keras.preprocessing.image import ImageDataGenerator
from keras.layers import Dense
from keras.layers import Dropout
from keras.layers import Flatten
from keras.layers import GlobalAveragePooling2D
from keras.callbacks import TensorBoard
from keras.constraints import maxnorm
from keras.optimizers import SGD
from keras.layers.convolutional import Conv2D
from keras.layers.convolutional import MaxPooling2D
from keras.utils import np_utils

from keras.applications import VGG19

import cv2

import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline

Using TensorFlow backend.


In [3]:
vgg19 = VGG19(include_top=False)

In [8]:
vgg19.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_2 (InputLayer)         (None, None, None, 3)     0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, None, None, 64)    1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, None, None, 64)    36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, None, None, 64)    0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, None, None, 128)   73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, None, None, 128)   147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, None, None, 128)   0         
__________

In [5]:
x = vgg19.output
x = GlobalAveragePooling2D()(x)
predictions = Dense(3, activation='softmax')(x)
predictions

<tf.Tensor 'dense_1/Softmax:0' shape=(?, 3) dtype=float32>

In [6]:
model = Model(vgg19.input, predictions)
model

In [13]:
# train just the custom layers
for layer in vgg19.layers:
    layer.trainable = False
    
model.compile(optimizer='rmsprop', loss='categorical_crossentropy', metrics=['accuracy'])

In [15]:
batch_size = 8

# this is the augmentation configuration we will use for training
train_datagen = ImageDataGenerator(
        rescale=1./255,
        shear_range=0.2,
        zoom_range=0.2,
        horizontal_flip=True)

# this is the augmentation configuration we will use for testing:
# only rescaling
val_datagen = ImageDataGenerator(rescale=1./255)

# this is a generator that will read pictures found in
# subfolers of 'data/train', and indefinitely generate
# batches of augmented image data
train_generator = train_datagen.flow_from_directory(
        'data/train/256x256_size/',  # this is the target directory
        target_size=(256, 256),  # made my images this size
        batch_size=batch_size,
        class_mode='categorical')  

# this is a similar generator, for validation data
validation_generator = val_datagen.flow_from_directory(
        'data/validation/256x256_size/',
        target_size=(256, 256),
        batch_size=batch_size,
        class_mode='categorical')


model.fit_generator(
        train_generator,
        steps_per_epoch=150 // batch_size,
        epochs=5,
        validation_data=validation_generator,
        validation_steps=30 // batch_size)

Found 150 images belonging to 3 classes.
Found 30 images belonging to 3 classes.
Epoch 1/5
12/12 [==============================] - 74s - loss: 1.0390 - acc: 0.5104 - val_loss: 0.9996 - val_acc: 0.5625
Epoch 2/5
12/12 [==============================] - 73s - loss: 0.9637 - acc: 0.6461 - val_loss: 0.9572 - val_acc: 0.5000
Epoch 3/5
12/12 [==============================] - 77s - loss: 0.9270 - acc: 0.5625 - val_loss: 0.8680 - val_acc: 0.8125
Epoch 4/5
12/12 [==============================] - 78s - loss: 0.8620 - acc: 0.7464 - val_loss: 0.8355 - val_acc: 0.6875
Epoch 5/5
12/12 [==============================] - 81s - loss: 0.8189 - acc: 0.7432 - val_loss: 0.8619 - val_acc: 0.5625


In [17]:
# at this point, the top layers are well trained and we can start fine-tuning
# convolutional layers from inception V3. We will freeze the bottom N layers
# and train the remaining top layers.

# let's visualize layer names and layer indices to see how many layers
# we should freeze:
for i, layer in enumerate(model.layers):
    print(i, layer.name)

# we chose to train the top 2 inception blocks, i.e. we will freeze
# the first 19 layers and unfreeze the rest:
for layer in model.layers[:19]:
    layer.trainable = False
for layer in model.layers[19:]:
    layer.trainable = True

0 input_2
1 block1_conv1
2 block1_conv2
3 block1_pool
4 block2_conv1
5 block2_conv2
6 block2_pool
7 block3_conv1
8 block3_conv2
9 block3_conv3
10 block3_conv4
11 block3_pool
12 block4_conv1
13 block4_conv2
14 block4_conv3
15 block4_conv4
16 block4_pool
17 block5_conv1
18 block5_conv2
19 block5_conv3
20 block5_conv4
21 block5_pool
22 global_average_pooling2d_1
23 dense_1


In [18]:
# we need to recompile the model for these modifications to take effect
# we use SGD with a low learning rate
model.compile(optimizer=SGD(lr=0.0001, momentum=0.9), loss='categorical_crossentropy', metrics=['accuracy'])

# we train our model again (this time fine-tuning the top 2 inception blocks
# alongside the top Dense layers
model.fit_generator(
        train_generator,
        steps_per_epoch=150 // batch_size,
        epochs=5,
        validation_data=validation_generator,
        validation_steps=30 // batch_size)

Epoch 1/5
18/18 [==============================] - 116s - loss: 0.8019 - acc: 0.7458 - val_loss: 0.7950 - val_acc: 0.7500
Epoch 2/5
18/18 [==============================] - 115s - loss: 0.7640 - acc: 0.7915 - val_loss: 0.7189 - val_acc: 0.7727
Epoch 3/5
18/18 [==============================] - 116s - loss: 0.7101 - acc: 0.7615 - val_loss: 0.7982 - val_acc: 0.6364
Epoch 4/5
18/18 [==============================] - 114s - loss: 0.6788 - acc: 0.7129 - val_loss: 0.6121 - val_acc: 0.8182
Epoch 5/5
18/18 [==============================] - 114s - loss: 0.6358 - acc: 0.7963 - val_loss: 0.6500 - val_acc: 0.7500


In [19]:
model.save('VGG19_trained.h5')

## how our model looks like

In [20]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_2 (InputLayer)         (None, None, None, 3)     0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, None, None, 64)    1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, None, None, 64)    36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, None, None, 64)    0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, None, None, 128)   73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, None, None, 128)   147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, None, None, 128)   0         
__________